In [26]:
import io
import pandas as pd
import requests
import pyarrow #Required to read parquet files

In [27]:
df = pd.read_parquet('data/yellow_tripdata_2024-01.parquet', engine= 'pyarrow')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [28]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,4


In [29]:
dim_datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
dim_datetime['pick_hour'] = df['tpep_pickup_datetime'].dt.hour
dim_datetime['pick_day'] = df['tpep_pickup_datetime'].dt.day
dim_datetime['pick_weekday'] = df['tpep_pickup_datetime'].dt.weekday
dim_datetime['pick_month'] = df['tpep_pickup_datetime'].dt.month
dim_datetime['pick_year'] = df['tpep_pickup_datetime'].dt.year

dim_datetime['drop_hour'] = df['tpep_dropoff_datetime'].dt.hour
dim_datetime['drop_day'] = df['tpep_dropoff_datetime'].dt.day
dim_datetime['drop_weekday'] = df['tpep_dropoff_datetime'].dt.weekday
dim_datetime['drop_month'] = df['tpep_dropoff_datetime'].dt.month
dim_datetime['drop_year'] = df['tpep_dropoff_datetime'].dt.year

dim_datetime['datetime_id'] = dim_datetime.index

dim_datetime.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_weekday,pick_month,pick_year,drop_hour,drop_day,drop_weekday,drop_month,drop_year,datetime_id
0,2024-01-01 00:57:55,2024-01-01 01:17:43,0,1,0,1,2024,1,1,0,1,2024,0
1,2024-01-01 00:03:00,2024-01-01 00:09:36,0,1,0,1,2024,0,1,0,1,2024,1
2,2024-01-01 00:17:06,2024-01-01 00:35:01,0,1,0,1,2024,0,1,0,1,2024,2
3,2024-01-01 00:36:38,2024-01-01 00:44:56,0,1,0,1,2024,0,1,0,1,2024,3
4,2024-01-01 00:46:51,2024-01-01 00:52:57,0,1,0,1,2024,0,1,0,1,2024,4


In [31]:
dim_passenger_count = df[['passenger_count']].reset_index(drop=True)
dim_passenger_count['passenger_count_id'] = dim_passenger_count.index

dim_passenger_count

,passenger_count,passenger_count_id
0,1.0,0
1,1.0,1
2,1.0,2
3,1.0,3
4,1.0,4
...,...,...
2964619,NaN,2964619
2964620,NaN,2964620
2964621,NaN,2964621
2964622,NaN,2964622


In [30]:
dim_trip_distance = df[['trip_distance']].reset_index(drop=True)
dim_trip_distance['trip_distance_id'] = dim_trip_distance.index

dim_trip_distance

,trip_distance,trip_distance_id
0,1.72,0
1,1.80,1
2,4.70,2
3,1.40,3
4,0.80,4
...,...,...
2964619,3.18,2964619
2964620,4.00,2964620
2964621,3.33,2964621
2964622,3.06,2964622


In [34]:

rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

dim_rate_code = df[['RatecodeID']].reset_index(drop=True)
dim_rate_code['rate_code_id'] = dim_rate_code.index
dim_rate_code['rate_code_name'] = dim_rate_code['RatecodeID'].map(rate_code_type)
dim_rate_code

,RatecodeID,rate_code_id,rate_code_name
0,1.0,0,Standard rate
1,1.0,1,Standard rate
2,1.0,2,Standard rate
3,1.0,3,Standard rate
4,1.0,4,Standard rate
...,...,...,...
2964619,NaN,2964619,NaN
2964620,NaN,2964620,NaN
2964621,NaN,2964621,NaN
2964622,NaN,2964622,NaN


In [37]:
dim_pickup_location = df[['PULocationID']].reset_index(drop=True)
dim_pickup_location['pickup_location_id'] = dim_pickup_location.index


dim_dropoff_location = df[['DOLocationID']].reset_index(drop=True)
dim_dropoff_location['dropoff_location_id'] = dim_dropoff_location.index
dim_dropoff_location

,DOLocationID,dropoff_location_id
0,79,0
1,236,1
2,79,2
3,211,3
4,148,4
...,...,...
2964619,263,2964619
2964620,236,2964620
2964621,25,2964621
2964622,13,2964622


In [39]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
dim_payment_type = df[['payment_type']].reset_index(drop=True)
dim_payment_type['payment_type_id'] = dim_payment_type.index
dim_payment_type['payment_type_name'] = dim_payment_type['payment_type'].map(payment_type_name)

dim_payment_type


,payment_type,payment_type_id,payment_type_name
0,2,0,Cash
1,1,1,Credit card
2,1,2,Credit card
3,1,3,Credit card
4,1,4,Credit card
...,...,...,...
2964619,0,2964619,NaN
2964620,0,2964620,NaN
2964621,0,2964621,NaN
2964622,0,2964622,NaN


In [41]:
fact_table = df.merge(dim_passenger_count, left_on='trip_id', right_on='passenger_count_id') \
             .merge(dim_trip_distance, left_on='trip_id', right_on='trip_distance_id') \
             .merge(dim_rate_code, left_on='trip_id', right_on='rate_code_id') \
             .merge(dim_pickup_location, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dim_dropoff_location, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(dim_datetime, left_on='trip_id', right_on='datetime_id') \
             .merge(dim_payment_type, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,0,0,17.70,1.00,0.5,0.00,0.00,1.0,22.70
1,1,1,1,1,1,1,N,1,1,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75
2,2,1,2,2,2,2,N,2,2,2,23.30,3.50,0.5,3.00,0.00,1.0,31.30
3,3,1,3,3,3,3,N,3,3,3,10.00,3.50,0.5,2.00,0.00,1.0,17.00
4,4,1,4,4,4,4,N,4,4,4,7.90,3.50,0.5,3.20,0.00,1.0,16.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2964619,2,2964619,2964619,2964619,2964619,None,2964619,2964619,2964619,15.77,0.00,0.5,2.00,0.00,1.0,21.77
2964620,2964620,1,2964620,2964620,2964620,2964620,None,2964620,2964620,2964620,18.40,1.00,0.5,2.34,0.00,1.0,25.74
2964621,2964621,2,2964621,2964621,2964621,2964621,None,2964621,2964621,2964621,19.97,0.00,0.5,0.00,0.00,1.0,23.97
2964622,2964622,2,2964622,2964622,2964622,2964622,None,2964622,2964622,2964622,23.88,0.00,0.5,5.58,0.00,1.0,33.46
